In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np

# Load the saved model and scaler
model = joblib.load("xgboost_loan_model.pkl")
scaler = joblib.load("scaler.pkl")

# Initialize Flask app
app = Flask(__name__)

@app.route('/predict', methods=['POST'])
def predict():
    # Parse input JSON
    input_data = request.get_json()

    # Ensure the input data contains all required features
    features = [
        "person_age", "person_gender", "person_education", "person_income",
        "person_emp_exp", "person_home_ownership", "loan_amnt", "loan_intent",
        "loan_int_rate", "loan_percent_income", "cb_person_cred_hist_length",
        "credit_score", "previous_loan_defaults_on_file"
    ]
    try:
        # Extract and reshape input features
        input_array = np.array([input_data[feature] for feature in features]).reshape(1, -1)

        # Scale the input data
        scaled_input = scaler.transform(input_array)

        # Get the prediction from the model
        prediction = model.predict(scaled_input)
        probability = model.predict_proba(scaled_input)[0][1]

        # Prepare response
        response = {
            "loan_status": "Approved" if prediction[0] == 1 else "Rejected",
            "approval_probability": probability
        }
        return jsonify(response)
    except KeyError as e:
        return jsonify({"error": f"Missing feature in input: {e}"}), 400

if __name__ == '__main__':
    app.run(debug=True)
